In [1]:
using Laplacians

objc[18980]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18980]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18980]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[18980]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_3/"

"../graphs/se_3/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [9]:
@time F = treeSolver(tree)
@time F(b);

 12.429566 seconds (2.81 M allocations: 183.971 MB, 0.76% gc time)
  0.389330 seconds (4.12 M allocations: 81.311 MB, 20.35% gc time)


In [12]:
# a run on standard Float64 data types
numIts = 1000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 1.7712969041872371e6
Working on iteration 20 with a-norm error 1.5187925011494306e6
Working on iteration 30 with a-norm error 914241.3878257711
Working on iteration 40 with a-norm error 600896.7036546426
Working on iteration 50 with a-norm error 415417.1398259632
Working on iteration 60 with a-norm error 315035.22720878856
Working on iteration 70 with a-norm error 215181.4288685037
Working on iteration 80 with a-norm error 167992.4373805947
Working on iteration 90 with a-norm error 140309.13491500143
Working on iteration 100 with a-norm error 115914.20984578966
Working on iteration 110 with a-norm error 101558.83934661775
Working on iteration 120 with a-norm error 86457.05687482444
Working on iteration 130 with a-norm error 74534.63333988967
Working on iteration 140 with a-norm error 68111.30316039355
Working on iteration 150 with a-norm error 60341.968910475654
Working on iteration 160 with a-norm error 54083.90959235182
Working on iteration 1

In [ ]:
numIts = 1000
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

In [ ]:
?pcgV

In [ ]:
typeof(laHP)